In [1]:
import site, os
try:
    print(site.getsitepackages())
except:
    print(os.path.dirname(site.__file__) + '/site-packages')

/Users/csaintje/Documents/Recherche/Python/Envs/scientific_3_5/bin/../lib/python3.5/site-packages


In [31]:
import numpy as np
import scipy.integrate as sci
import scipy.optimize as sco
import scipy.stats as scs
import itertools as it
import matplotlib
%matplotlib qt4
#print (matplotlib.rcsetup.interactive_bk)
#print (matplotlib.rcsetup.non_interactive_bk)
#print (matplotlib.rcsetup.all_backends)
print (matplotlib.get_backend())
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import joblib
from joblib import Parallel, delayed

Qt4Agg


In [42]:
import theano.tensor as T
import theano

INFO (theano.gof.compilelock): Waiting for existing lock by process '24217' (I am process '27129')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/csaintje/.theano/compiledir_Darwin-15.4.0-x86_64-i386-64bit-i386-3.5.1-64/lock_dir


Details for gaussian distribution as an exponential family

In [3]:
def lb2th(mu,sigma2):
    return mu / sigma2, 0.5 / sigma2

def lb2et(mu, sigma2):
    return mu, -(mu**2+sigma2)

def th2lb(th1,th2):
    return 0.5*th1/th2, 0.5 / th2

def th2et(th1,th2):
    return 0.5*th1/th2, 0.5*(-0.5*th1**2/th2 - 1) / th2

def et2lb(et1,et2):
    return et1, -(et1**2+et2)

def et2th(et1,et2):
    return - et1 /(et1**2 + et2), -0.5 /(et1**2 + et2)

def s_1(x):
    return x

def s_2(x):
    return -x*x

def F_1D(theta1,theta2):
    return 0.25*theta1*theta1/theta2 + 0.5*np.log(np.pi) - 0.5*np.log(theta2) 

def gradF_1_1D(theta_1,theta_2):
    return 0.5*theta_1/theta_2

def gradF_2_1D(theta_1, theta_2):
    temp_1 = 0.5 / theta_2
    temp_2 = temp_1 * theta_1
    return -1. * (temp_2 * temp_2 + temp_1)

def gradF_1_nD(theta_1,theta_2):
    return 0.5*np.dot(np.inv(theta_2), theta_1)

def gradF_2_nD(theta_1, theta_2):
    temp_1 = 0.5*np.inv(theta_2)
    temp_2 = np.dot(temp1,theta_1)
    return - np.outer(temp_2,temp_2) - temp_1

def gradG_1_1D(eta_1,eta_2):
    return eta1 / (-eta_1 * eta1 - eta2)

def gradG_2_1D(eta_1,eta_2):
    return 0.5  / (-eta_1 * eta1 - eta2)

def gradG_1_nD(eta_1,eta_2):
    return np.dot(np.inv(-np.outer(eta_1, eta1) - eta2), eta1)

def gradG_2_nD(eta_1,eta_2):
    return 0.5*np.inv(-np.outer(eta_1, eta1) - eta2)


Cas 1D - Data

In [17]:
seed = 13
np.random.seed(seed)
N, batch_size = 2000, 100
mu_true, sigma_true = 1, 2
sigma2_true = sigma_true**2
theta1_true, theta2_true = lb2th(mu_true, sigma2_true)
eta1_true, eta2_true = lb2et(mu_true, sigma2_true)
X = np.random.normal(mu_true,np.sqrt(sigma2_true), N)
replicate = 5
mu_0, sigma2_0 = 0, 10
a_0 = 1 
lb_init = [(mu, sigma2) for mu, sigma2 in zip(np.random.normal(mu_0,np.sqrt(sigma2_0), replicate),
                                                  scs.invgamma.rvs(a_0, size=replicate))]
th_init = [lb2th(mu, sigma2) for mu, sigma2 in lb_init]
et_init = [lb2et(mu, sigma2) for mu, sigma2 in lb_init]
#print(lb_init, th_init, et_init)
def batches(Y, bs):
    nb = np.int(np.ceil(len(Y) / bs))
    Yb = [Y[i * bs : (i+1) * bs] for i in range(nb)]
    def __temp(i):
        return Yb[i]
    return nb, Yb, __temp

Xt = X[: N//4]
Nt = len(Xt)
NtB, XtB, XtB_f = batches(Xt, batch_size)
Xv = X[N//4 : N//2]
Nv = len(Xv)
NvB, XvB, XvB_f = batches(Xv, batch_size)
Xtest = X[N//2:]

In [21]:
def ll(x, mu, sigma2):
    return -(x - mu)**2 /(2 * sigma2) - np.log(np.sqrt(2 * sigma2 * np.pi)) 

def ll_theta(x, theta1, theta2):
    return s_1(x) * theta1 + s_2(x) * theta2 - F_1D(theta1, theta2)

def ll_eta(x, eta1, eta2):
    theta1 = - eta1 / (eta1*eta1 + eta2)
    theta2 = - 0.5  / (eta1*eta1 + eta2)
    return ll_theta(x, theta1, theta2)
    

def ave_ll(mu, sigma2, chi):
    N = len(chi)
    return (1. / N) * sum(ll(x, mu, sigma2) for x in chi)
    
def C_N(mu, sigma2, chi):
    return -ave_ll(mu, sigma2, chi)

np.testing.assert_allclose(ll(4, mu_true, sigma2_true),
                          ll_theta(4, theta1_true, theta2_true))
np.testing.assert_allclose(ll(4, mu_true, sigma2_true),
                          ll_eta(4, eta1_true, eta2_true))

In [ ]:
class ListTable(list):
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            for col in row:
                html.append("<td>{0}</td>".format(col))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
def disp_results(results):
    n = len(results)
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 5))
    colors = plt.cm.jet(np.linspace(0, 1, n))
    table = ListTable()
    table.append(['', 'Train', 'Test', 'mu', 'sigma^2'])
    table.append(['True', ave_ll(mu_true, sigma2_true, Xt), 
                  ave_ll(mu_true, sigma2_true, Xtest),
                  str(mu_true), str(sigma_true**2)])
    for c, result in enumerate(results):
        train, test, mu, sig, method = result
        axes[0,0].plot(train, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[0,0].set_title('Average likelihood on train set')
        axes[0,0].legend(loc=4)
        axes[0,1].plot(test, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[0,1].set_title('Average likelihood on test set')
        axes[0,1].legend(loc=4)
        axes[1,0].plot(mu, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[1,0].set_title('Estimates of $\mu$')
        axes[1,0].legend(loc=4)
        axes[1,1].plot(sig, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[1,1].set_title('Estimates of $\sigma^2$')
        axes[1,1].legend(loc=4)
        table.append([method, train[-1], test[-1],mu[-1], sig[-1]])
    display(HTML(table._repr_html_()))

In [ ]:
mem = joblib.Memory(cachedir='/tmp/joblib')

import Parallel, delayed
>>> from math import sqrt
>>> Parallel(n_jobs=1)(delayed(sqrt)(i**2) for i in range(10))
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

def result_results(fun, **kwargs):
    
    mem = Memory(cachedir='/tmp/joblib')
    

from joblib import Memory
>>> mem = Memory(cachedir='/tmp/joblib')
>>> import numpy as np
>>> a = np.vander(np.arange(3)).astype(np.float)
>>> square = mem.cache(np.square)
>>> b = square(a)               

# Points stationnaires de $C(\theta) = \mathbb{E}_{\pi} [C(\theta,x)] = \mathbb{E}_{\pi} [-\log p(x;\theta)]$ par Robbins-Monro


### Robbins-Monro

On dispose d'une fonction inconnue (supposée monotone) $M(\theta)$ telle que 
$$M(\theta) = \mathbb{E}_{\pi(\beta|\theta)} [\beta]$$ 
avec $\beta$ une v.a désignant des observations bruitées de $M(\theta)$.

On cherche la valeur $\theta^*$ telle que $M(\theta^*) = \alpha$.

Suite convergente de Robbins-Monro : $$\theta^{(t+1)} - \theta^{(t)} = a^{(t)} (\alpha - \beta^{(t)})$$

### Robbins-Monro  et gradient stochastique

$M(\theta) := \nabla_{\theta} C(\theta)$ est le gradient d'une fonction inconnue $C$. 

On cherche la valeur $\theta^*$ telle que $M(\theta^*) = \nabla C(\theta^*) = 0$.

Suite convergente de Robbins-Monro : $$\theta^{(t+1)} = \theta^{(t)} - a^{(t)} \beta^{(t)}$$
où $\beta^{(t)}$ est une observation bruitée de $\nabla_{\theta} C(\theta^{(t)})$.

### Pour notre cas
La fonction à minimiser est:
$$C(\theta) = \mathbb{E}_{\pi} [- \log p(x;\theta)]$$
où $\pi$ est la distribution inconnue dont on cherche une approximation $p$ paramétrée par $\theta$ (identique à la minimisation sur $\theta$ de $KL(\pi || p(.;\theta))$)

Son équivalent en discret:
$$C_N(\theta) = - N^{-1} \sum_i \log p(x_i;\theta)$$
avec $\lim_{N \rightarrow +\infty} C_N(\theta) = C(\theta)$ 

Sous conditions de regularité et dans la famille exponentielle, 
$$\nabla_{\theta} C(\theta) = \mathbb{E}_{\pi} [- \nabla_{\theta}\log p(x;\theta)]  = \mathbb{E}_{\pi} [- s(x) + \nabla_{\theta} F (\theta)]$$

Pour le relier à Robbin-Monro, on a une observation bruitée
$$\beta^{(t)} = - s(x^{(t)}) + \nabla_{\theta} F (\theta^{(t)})$$
et donc la suite convergente $$\theta^{(t+1)} = \theta^{(t)} + a^{(t)} (s(x^{(t)}) - \nabla_{\theta} F (\theta^{(t)}))$$

Est ce que la formulation suivante est équivalente ? sans doute non ...
$$\eta^{(t+1)} = \eta^{(t)} + a^{(t)} (s(x^{(t)}) - \eta^{(t)}))$$

Dans l'espace $H$ (paramètre d'espérance), la même optimisation:
$$C(\eta) = \mathbb{E}_{\pi} [- \log p(x;\eta)] = \mathbb{E}_{\pi} [B_{F^*}(s(x) : \eta) - F^*(s(x)) - k(x)]$$
$$\nabla_{\eta} C(\eta) = \mathbb{E}_{\pi} [\nabla_{\eta} B_{F^*}(s(x) : \eta)] = \mathbb{E}_{\pi} [\nabla_{\eta} (F^*(s(x)) - F^*(\eta) - <s(x) - \eta, \nabla_\eta F^*(\eta)>)]$$
$$ = \mathbb{E}_{\pi} [ - \nabla_{\eta} F^*(\eta) - [-1,0;0,-1]*\nabla_\eta F^*(\eta) - Hess F^*(\eta) (s(x) - \eta)]$$
$$ = \mathbb{E}_{\pi} [ - \nabla_{\eta} F^*(\eta) + \nabla_{\eta} F^*(\eta) - Hess F^*(\eta) (s(x) - \eta)] = \mathbb{E}_{\pi} [- Hess F^*(\eta) (s(x) - \eta)]$$
et donc la suite convergente 
$$\eta^{(t+1)} = \eta^{(t)} + a^{(t)} Hess F^*(\eta^{(t)}) (s(x^{(t)}) - \eta^{(t)})$$

In [ ]:
from sympy import Function, Derivative, var, simplify
from sympy.abc import x, y, z, t
from sympy import diff, log, pi
s1 = Function("s1")(x,y)
s2 = Function("s2")(x,y)
F = Function("F")(z,t)
F1 = Derivative(F, z)
F2 = Derivative(F, t)
expr = -((s1 - z) * F1 + (s2 - t)*F2)
print (diff(expr, z))
print (diff(expr, t))
x, eta1, eta2 = var('x eta1 eta2')
s1 = x
s2 = -x*x
ld = log(2*(- eta1*eta1 - eta2))
F =  - 0.5 * (1 + log(pi) + ld)
dF1 = diff(F, eta1)
dF2 = diff(F, eta2)
dF11 = diff(dF1, eta1)
dF12 = diff(dF1, eta2)
dF21 = diff(dF2, eta1)
dF22 = diff(dF2, eta2)
print (dF11)
print (dF12)
print (dF21)
print (dF22)
print (-simplify(dF11*(s1-eta1)+dF12*(s2-eta2)))
print (-simplify(dF21*(s1-eta1)+dF22*(s2-eta2)))

# Debut des manips

In [23]:
results = []

## Points stationnaires de $C_N(\theta)$ par dérivation exacte

   $$\nabla C_N(\theta) = 0 \equiv -N^{-1} \sum_i (s(x_i) - \nabla F(\theta)) = 0 \equiv  \nabla F(\theta) = N^{-1} \sum_i s(x_i)$$

In [ ]:
gradF_pt_stat_1 = np.sum(s_1(x) for x in Xt) / Nt
gradF_pt_stat_2 = np.sum(s_2(x) for x in Xt) / Nt
pt_stat = gradF_pt_stat_1, - gradF_pt_stat_1**2 - gradF_pt_stat_2
print ((mu_true, sigma2_true), ' vs ', pt_stat)
print('average ll on training set : ', 
      ave_ll(mu_true, sigma2_true, Xt), ' vs ',
      ave_ll(pt_stat[0], pt_stat[1], Xt))
print('average ll on test set : ', 
      ave_ll(mu_true, sigma2_true, Xtest), ' vs ',
      ave_ll(pt_stat[0], pt_stat[1], Xtest))
gradF_pt_stat_1 = np.cumsum([s_1(x) for x in Xt]) / np.arange(1,Nt+1)
gradF_pt_stat_2 = np.cumsum([s_2(x) for x in Xt]) / np.arange(1,Nt+1)
mu_list = gradF_pt_stat_1
sig_list = -gradF_pt_stat_1**2 -gradF_pt_stat_2
ave_ll_train = [ave_ll(mu, sig, Xt) for mu,sig in zip(mu_list, sig_list)]
ave_ll_test = [ave_ll(mu, sig, Xtest) for mu,sig in zip(mu_list, sig_list)]
this_result = [ave_ll_train, ave_ll_test, mu_list, sig_list, 'Exact']
results.append(this_result)

In [ ]:
disp_results([this_result])

# Optimization via scipy

In [ ]:
def fun_C_N(mu_sigma2):
    return C_N(mu_sigma2[0], mu_sigma2[1], Xt)
np.random.rand(seed)
x0 = (np.random.randn(), np.random.random())
bnds = ((-np.inf, np.inf), (1e-6, np.inf))   # variance is positive
res_C_N = sco.minimize(fun_C_N, x0, bounds=bnds) #, options={'gtol': 1e-6, 'disp': True})
print(res_C_N)
print('average ll on training set : ', 
      ave_ll(mu_true, sigma2_true, Xt), ' vs ',
      ave_ll(res_C_N.x[0], res_C_N.x[1], Xt))
print('average ll on test set : ', 
      ave_ll(mu_true, sigma2_true, Xtest), ' vs ',
      ave_ll(res_C_N.x[0], res_C_N.x[1], Xtest))

## Optimization via Stochastic Gradient Descent  (natural space)
mostly fail:
- $\alpha = 0.01$

success:
- $\alpha = 0.001$

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10

mu_0, sigma2_0 = np.random.randn(), np.random.random()
theta1, theta2 = mu_0 / sigma2_0, 0.5 / sigma2_0

def grad_nll_1(x, theta1, theta2):
    return - (s_1(x) - gradF_1_1D(theta1, theta2))

def grad_nll_2(x, theta1, theta2):
    return - (s_2(x) - gradF_2_1D(theta1, theta2))
    
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for __ in range(epochs):
    for xt in Xt:
        #print("avant theta",theta1, theta2, " x ", xt)
        #print ("grad 1 ", gradF_1_1D(theta1, theta2), grad_nll_1(xt, theta1, theta2))
        #print ("grad 2 ", gradF_2_1D(theta1, theta2), grad_nll_2(xt, theta1, theta2))
        theta1 -= alpha*grad_nll_1(xt, theta1, theta2)
        theta2 -= alpha*grad_nll_2(xt, theta1, theta2)
        #print("apres theta", theta1, theta2)
        mu_est, sigma2_est = 0.5*theta1/theta2, 0.5/theta2
        #print("apres mu_sd2",   mu_est, sigma2_est)
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest)) 
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(T)-'+str(alpha)]
results.append(this_result)

In [25]:
#disp_results([this_result])
disp_results(results)

NameError: name 'disp_results' is not defined

## Optimization via Stochastic Gradient Descent  

Il s'agit d'un réécriture simple en remplacant $\nabla F$ par $\eta$. 

Cela ressemble dans la forme à l'approximation stochastique du Online EM.

L'algorihme converge car les deux optimisations (avec celles du dessus) sont liées. 

(J'ai fait le calcul $\eta^{(n+1)} - \eta^{(n)}$ vers $\theta^{(n+1)} - \theta^{(n)}$)

Il ne correspond pas à un SGD dand $H$.

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10
mu_0, sigma2_0 = np.random.randn(), np.random.random()
eta1, eta2 = mu_0 , -(mu_0*mu_0 + sigma2_0)
def grad_nll_1(x, eta1, eta2):
    return -(s_1(x) - eta1)

def grad_nll_2(x, eta1, eta2):
    return -(s_2(x) - eta2)
    
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for __ in range(epochs):
    for x in Xt:
        #print("avant eta",eta1, eta2, " x ", x)
        #print ("grad 1 ", grad_nll_1(x, eta1, eta2))
        #print ("grad 2 ", grad_nll_2(x, eta1, eta2))
        eta1 -= alpha*grad_nll_1(x, eta1, eta2)
        eta2 -= alpha*grad_nll_2(x, eta1, eta2)
        #print("apres eta", eta1, eta2)
        mu_est, sigma2_est = eta1, -(eta1*eta1+eta2)
        #print("apres mu_sd2", mu_est, sd_est)
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(M)-'+str(alpha)]
results.append(this_result)

In [ ]:
#disp_results([this_result])
disp_results(results)

## Optimization via Stochastic Gradient Descent  (Expectation space)

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10
mu_0, sigma2_0 = np.random.randn(), np.random.random()
eta1, eta2 = mu_0 , -(mu_0*mu_0 + sigma2_0)
def grad_1(x, eta1, eta2):
    return (1.0*eta1*(eta2 + x**2) + (eta1 - x)*(8.0*eta1**2 - 8.0*eta2)/8.)/(eta1**2 + eta2)**2

def grad_2(x, eta1, eta2):
    return (1.0*eta1*(eta1 - x) + 0.5*eta2 + 0.5*x**2)/(eta1**2 + eta2)**2
    
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for __ in range(epochs):
    for x in Xt:
        #print("avant eta",eta1, eta2, " x ", x)
        #print ("grad 1 ", grad_nll_1(x, eta1, eta2))
        #print ("grad 2 ", grad_nll_2(x, eta1, eta2))
        eta1 -= alpha*grad_1(x, eta1, eta2)
        eta2 -= alpha*grad_2(x, eta1, eta2)
        #print("apres eta", eta1, eta2)
        mu_est, sigma2_est = eta1, -(eta1*eta1+eta2)
        #print("apres mu_sd2", mu_est, sd_est)
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(H)-CSJ'+str(alpha)]
results.append(this_result)

In [ ]:
#disp_results([this_result])
disp_results(results[2:])

# Theano

In [39]:
def train_on_lb(mu_sigma2, update_method, **kwargs):
    mu_0, sigma2_0 = mu_sigma2
    x = T.scalar()
    mu = theano.shared(mu_0, name="mu")
    sigma2 = theano.shared(sigma2_0, name="sigma2")
    cost_lb =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))
    params = [mu, sigma2]
    updates = update_method(cost_lb, params, **kwargs)
    aux = False
    if len(updates) > 1:
        aux_est, updates = updates
        aux = True
    train_lb = theano.function(inputs=[x], updates=updates)
    mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
    for xt in Xt:
        train_lb(xt)
        if aux:
            mu_est, sigma2_est = aux_est[0].get_value(), aux_est[1].get_value()
        else:
            mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
    return [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list]

def train_on_th(th1_th2, update_method, **kwargs):
    th1_0, th2_0 = th1_th2
    x = T.scalar()
    theta1 = theano.shared(th1_0, name="theta1")
    theta2 = theano.shared(th2_0, name="theta2")
    cost_th =  -(x*theta1 - theta2*x*x - 0.25*theta1*theta1/theta2 - 0.5*np.log(np.pi) +0.5*T.log(theta2))
    params = [theta1, theta2]
    train_th = theano.function(inputs=[x], updates=update_method(cost_th, params, **kwargs))
    mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
    for xt in Xt:
        train_th(xt)
        mu_est, sigma2_est = th2lb(theta1.get_value(), theta2.get_value())
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
    return [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list]

def train_on_et(et1_et2, update_method, **kwargs):
    et1_0, et2_0 = et1_et2
    x = T.scalar()
    eta1 = theano.shared(et1_0, name="eta1")
    eta2 = theano.shared(et2_0, name="eta2")
    theta1 = - eta1 / (eta1*eta1 + eta2)
    theta2 = - 0.5  / (eta1*eta1 + eta2)
    cost_et =  -(x*theta1 - theta2*x*x - 0.25*theta1*theta1/theta2 - 0.5*np.log(np.pi) +0.5*T.log(theta2))
    params = [eta1, eta2]
    train_et = theano.function(inputs=[x], updates=update_method(cost_et, params, **kwargs))
    mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
    for xt in Xt:
        train_th(xt)
        mu_est, sigma2_est = et2lb(eta1.get_value(), eta2.get_value())
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
    return [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list]

In [40]:
mem = joblib.Memory(cachedir='/tmp/joblib')
train_on_lb_cache = mem.cache(train_on_lb)
train_on_th_cache = mem.cache(train_on_th)
train_on_et_cache = mem.cache(train_on_et)

In [29]:
def gradient_updates(cost, params, alpha):
    updates = [(param, param - alpha*T.grad(cost, param)) for param in params]
    return updates

def gradient_updates_momentum(cost, params, alpha, momentum):
    """
    http://caffe.berkeleyvision.org/tutorial/solver.html
    """
    assert momentum < 1 and momentum >= 0
    updates = []
    for param in params:
        V = theano.shared(param.get_value()*0.)
        updates.append((param, param + V))
        updates.append((V, momentum*V - alpha * T.grad(cost, param)))
    return updates

# on peut rajouter d'autres méthodes ici

def gradient_updates_average(cost, params, alpha, t0):
    # a verifier
    updates = []
    ave_est = []
    t = theano.shared(0.)
    updates.append((t, t+1))   
    for param in params:
        ave = theano.shared(0.)
        ave_est.append(ave)
        updates.append((ave, (t <= t0) * (param - alpha*T.grad(cost, param)) + 
                        (t > t0) * ((t-t0)*ave + (param - alpha*T.grad(cost, param)))/(t-t0+1)))
        updates.append((param, param - alpha*T.grad(cost, param)))
    return updates, ave_est

def gradient_updates_adam(cost, params, alpha, beta1, beta2):
    assert beta1 < 1 and beta1 >= 0
    assert beta2 < 1 and beta2 >= 0
    updates = []
    t = theano.shared(1.)
    updates.append((t, t+1))
    for param in params:
        gt = T.grad(cost, param) 
        mt = theano.shared(0.)
        updates.append((mt, beta1 * mt + (1-beta1) * gt))
        vt = theano.shared(0.)
        updates.append((vt, beta2 * vt + (1-beta2) * gt * gt))
        alpha_t = theano.shared(0.)
        updates.append((alpha_t, 
                        alpha*T.sqrt(1 - beta2**t)/(1 - beta1**t)))
        updates.append((param, param - alpha_t*mt/(T.sqrt(vt + 1e-8))))
    return updates   

## Optimization SGD via Theano (source space)

In [41]:
this_result = train_on_lb(lb_init[0], gradient_updates, alpha=0.001)
this_result2 = train_on_lb_cache(lb_init[0], gradient_updates, alpha=0.001)
this_result3 = train_on_lb_cache(lb_init[0], gradient_updates, alpha=0.001)

NameError: name 'T' is not defined

In [38]:
alphas = [0.1, 0.05, 0.01, 0.005, 0.001]
results_SGD_source = [train_on_lb_cache(lb_init[0], gradient_updates, alpha=alpha) for alpha in alphas] 

ValueError: Wrong number of arguments for train_on_lb(Xt, mu_sigma2, update_method, **kwargs):
     train_on_lb((-1.2522472864061529, 0.60546517297663849), <function gradient_updates at 0x11217b0d0>, alpha=0.1) was called.

## Optimization SGD via Theano (source space)

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10
# Declare Theano symbolic variables
x = T.scalar()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates(nll, [mu, sigma2], alpha))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        __ = train(xt)
        mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest)) 
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(S)-'+str(alpha)]
results.append(this_result)

# Optimization SGD via Theano (natural space)

Fails for $\alpha =0.005$

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10

# Declare Theano symbolic variables
x = T.scalar()

mu_0, sigma2_0 = np.random.randn(), np.random.random() 
theta1 = theano.shared(mu_0 / sigma2_0, name="theta1") # ensure initial value is positive
theta2 = theano.shared(0.5 / sigma2_0, name="theta2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll_theta =  -(x*theta1 - theta2*x*x - 0.25*theta1*theta1/theta2 - 0.5*np.log(np.pi) +0.5*T.log(theta2))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates(nll_theta, [theta1, theta2], alpha))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        mu_est = 0.5*theta1.get_value()/theta2.get_value()
        sigma2_est = 0.5/theta2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(T)-'+str(alpha)]
results.append(this_result)

# Optimization SGD via Theano (expectation space)

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10

# Declare Theano symbolic variables
x = T.scalar()

mu_0, sigma2_0 = np.random.randn(), np.random.random()
eta1 = theano.shared(mu_0, name="eta1") # ensure initial value is positive
eta2 = theano.shared(-(mu_0*mu_0 + sigma2_0), name="eta2") # ensure initial value is positive
print(eta1.get_value(), eta2.get_value())
# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
theta1 = - eta1 / (eta1*eta1 + eta2)
theta2 = - 0.5  / (eta1*eta1 + eta2)
nll_eta =  -(x*theta1 - theta2*x*x - 0.25*theta1*theta1/theta2 - 0.5*np.log(np.pi) +0.5*T.log(theta2))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates(nll_eta, [eta1, eta2], alpha))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        # print(g1, s_1(xt) - eta1.get_value())
        # print(g2, s_2(xt) - eta2.get_value())
        mu_est = eta1.get_value()
        sigma2_est = - (eta1.get_value()*eta1.get_value()+eta2.get_value())
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(H)- Theano '+str(alpha)]
results.append(this_result)

## Optimization via Theano +  SGD Nesterov momemtum (Source space)

In [ ]:
np.random.seed(seed)

alpha = 0.01
momentum = 0.9
epochs = 10
# Declare Theano symbolic variables
x = T.scalar()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates_momentum(nll, [mu, sigma2], alpha, momentum))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'Mom. SGD(S)-'+str(momentum)+','+str(alpha)]
results.append(this_result)

## Optimization via Theano +  ADAM (Source space)

In [ ]:
np.random.seed(seed)

alpha = 0.005
beta1 = 0.9
beta2 = 0.995
epochs = 10
# Declare Theano symbolic variables
x = T.scalar()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates_adam(nll, [mu, sigma2], 
                                        alpha, beta1, beta2))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest)) 
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'Adam(S)-'+str(beta1)+','+str(beta2)+','+str(alpha)]
results.append(this_result)

In [ ]:
disp_results(results)